对手写的0~9数字进行分类

In [44]:
%matplotlib inline
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import random 
from scipy.io import loadmat
import scipy.io
import matplotlib.cm as cm
from scipy.special import expit
from PIL import Image

In [45]:
data=loadmat('ex3data1.mat')
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

# sigmod函数

In [46]:
def sigmoid(z):
    return 1/(1+np.exp(-z))

代价函数：
$$J\left( \theta  \right)=\frac{1}{m}\sum\limits_{i=1}^{m}{[-{{y}^{(i)}}\log \left( {{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)-\left( 1-{{y}^{(i)}} \right)\log \left( 1-{{h}_{\theta }}\left( {{x}^{(i)}} \right) \right)]}+\frac{\lambda }{2m}\sum\limits_{j=1}^{n}{\theta _{j}^{2}}$$

Tip：可以通过np.matrix()函数将一个变量转换为numpy型矩阵

In [47]:
def cost(theta,X,y,learningRate):
    #将theta，X，y转换为numpy类型的矩阵
    theta=np.matrix(theta)
    X=np.matrix(X)
    y=np.matrix(y)

    #损失函数
    cross_cost=np.multiply(-y,np.log(sigmoid(X*theta.T)))-np.multiply((1-y),np.log(1-sigmoid(X*theta.T)))
    # 正则化部分计算
    reg=(learningRate/(2*len(X)))*np.sum(np.power(theta[:,1:theta.shape[1]],2))

    # 计算总损失
    whole_cost=np.sum(cross_cost)/len(X)+reg
    return whole_cost


如果我们要使用梯度下降法令这个代价函数最小化，因为我们未对${{\theta }_{0}}$ 进行正则化，所以梯度下降算法将分两种情形：
\begin{align}
  & Repeat\text{ }until\text{ }convergence\text{ }\!\!\{\!\!\text{ } \\ 
 & \text{     }{{\theta }_{0}}:={{\theta }_{0}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{_{0}}^{(i)}} \\ 
 & \text{     }{{\theta }_{j}}:={{\theta }_{j}}-a\frac{1}{m}\sum\limits_{i=1}^{m}{[{{h}_{\theta }}\left( {{x}^{(i)}} \right)-{{y}^{(i)}}]x_{j}^{(i)}}+\frac{\lambda }{m}{{\theta }_{j}} \\ 
 & \text{          }\!\!\}\!\!\text{ } \\ 
 & Repeat \\ 
\end{align}


In [48]:
def gradient(theta,X,y,learningRate):
    theta=np.matrix(theta)
    X = np.matrix(X)
    y = np.matrix(y)

    # 将theta矩阵转换成列向量
    parameters=int(theta.ravel().shape[1])

    error = sigmoid(X * theta.T) - y
    grad=((X.T*error)/len(X)).T+((learningRate / len(X)) * theta)

    grad[0,0]=np.sum(np.multiply(error,X[:,0]))/len(X)

    return np.array(grad).ravel()


现在我们已经定义了代价函数和梯度函数，现在是构建分类器的时候了。 对于这个任务，我们有10个可能的类，并且由于逻辑回归只能一次在2个类之间进行分类，我们需要多类分类的策略。 在本练习中，我们的任务是实现一对一全分类方法，其中具有k个不同类的标签就有k个分类器，每个分类器在“类别 i”和“不是 i”之间决定。 我们将把分类器训练包含在一个函数中，该函数计算10个分类器中的每个分类器的最终权重，并将权重返回为k X（n + 1）数组，其中n是参数数量。

In [49]:
from scipy.optimize import minimize

def one_vs_all(X, y, num_labels, learning_rate):
    rows = X.shape[0]
    params = X.shape[1]

    #k X (n + 1) 大小的参数矩阵  k=10  n+1=401  构建10个逻辑回归分类器
    all_theta=np.zeros((num_labels,params+1))

    #axis=1表示按列进行操作
    X=np.insert(X,0,values=np.ones(rows),axis=1)


    for i in range(1, num_labels + 1):
        theta = np.zeros(params + 1)
        y_i = np.array([1 if label == i else 0 for label in y])
        y_i = np.reshape(y_i, (rows, 1))  #变成一个列向量
        
        # minimize the objective function
        fmin = minimize(fun=cost, x0=theta, args=(X, y_i, learning_rate), method='TNC', jac=gradient)
        all_theta[i-1,:] = fmin.x
    
    return all_theta

In [50]:
#赋值
rows=data['X'].shape[0]
params=data['X'].shape[1]

all_theta=np.zeros((10,params+1))
X=np.insert(data['X'],0,values=np.ones(rows),axis=1)
theta=np.zeros(params+1)
y_0 = np.array([1 if label == 0 else 0 for label in data['y']])
y_0 = np.reshape(y_0, (rows, 1))

X.shape, y_0.shape, theta.shape, all_theta.shape


((5000, 401), (5000, 1), (401,), (10, 401))

theta是一维数组，因此当它被转换为计算梯度的代码中的矩阵时，它变为（1×401）矩阵。 我们还检查y中的类标签，以确保它们看起来像我们想象的一致。

In [51]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [52]:
all_theta = one_vs_all(data['X'], data['y'], 10, 1)
all_theta

array([[-2.38291346e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30474743e-03, -8.24531507e-10,  0.00000000e+00],
       [-3.18403111e+00,  0.00000000e+00,  0.00000000e+00, ...,
         4.46025411e-03, -5.08531235e-04,  0.00000000e+00],
       [-4.80012163e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -2.88116417e-05, -2.47992614e-07,  0.00000000e+00],
       ...,
       [-7.98722644e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -8.95295280e-05,  7.21832616e-06,  0.00000000e+00],
       [-4.57235036e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.33349450e-03,  9.96261157e-05,  0.00000000e+00],
       [-5.40509120e+00,  0.00000000e+00,  0.00000000e+00, ...,
        -1.16616386e-04,  7.88424336e-06,  0.00000000e+00]])

In [53]:
def predict_all(X, all_theta):
    # INPUT：参数值theta，测试数据X
    # OUTPUT：预测值
    # TODO：对测试数据进行预测
    
    # STEP1：获取矩阵的维度信息
    rows = X.shape[0]
    params = X.shape[1]
    num_labels = all_theta.shape[0]
    
    # STEP2：把矩阵X加入一行零元素
    # your code here  (appro ~ 1 lines)
    X = np.insert(X, 0, values=np.ones(rows), axis=1)
    
    # STEP3：把矩阵X和all_theta转换为numpy型矩阵
    # your code here  (appro ~ 2 lines)
    X = np.matrix(X)
    all_theta = np.matrix(all_theta)
    
    # STEP4：计算样本属于每一类的概率
    # your code here  (appro ~ 1 lines)
    h = sigmoid(X * all_theta.T)
    
    # STEP5：找到每个样本中预测概率最大的值  存储的是每一行中值最大的列坐标
    # your code here  (appro ~ 1 lines)
    h_argmax = np.argmax(h, axis=1)
    
    # STEP6：因为我们的数组是零索引的，所以我们需要为真正的标签+1
    h_argmax = h_argmax + 1
    
    return h_argmax

In [54]:
y_pred=predict_all(data['X'],all_theta)
y_pred
correct = [1 if a == b else 0 for (a, b) in zip(y_pred, data['y'])]
accuracy = (sum(map(int, correct)) / float(len(correct)))
print ('accuracy = {0}%'.format(accuracy * 100))

accuracy = 94.46%


In [1]:
def getDatumImg(row):
    """
    Function that is handed a single np array with shape 1x400,
    crates an image object from it, and returns it
    """
    width, height = 20, 20
    square = row[1:].reshape(width,height)
    return square.T
    
def displayData(indices_to_display = None):
    """
    Function that picks 100 random rows from X, creates a 20x20 image from each,
    then stitches them together into a 10x10 grid of images, and shows it.
    """
    width, height = 20, 20
    nrows, ncols = 10, 10
    if not indices_to_display:
        indices_to_display = random.sample(range(X.shape[0]), nrows*ncols)
        
    big_picture = np.zeros((height*nrows,width*ncols))
    
    irow, icol = 0, 0
    for idx in indices_to_display:
        if icol == ncols:
            irow += 1
            icol  = 0
        iimg = getDatumImg(X[idx])
        big_picture[irow*height:irow*height+iimg.shape[0],icol*width:icol*width+iimg.shape[1]] = iimg
        icol += 1
    fig = plt.figure(figsize=(6,6))
    img = scipy.misc.toimage( big_picture )
    plt.imshow(img,cmap = cm.Greys_r)